In [359]:
import sys
import time
import math
import binascii
import argparse
import subprocess
from subprocess import Popen, PIPE
import os
import psutil
import time
import multiprocessing
import glob
import pandas as pd
import json
import warnings
import ast
import multiprocessing
from gtfparse import read_gtf
from io import BytesIO, StringIO
import numpy as np
import random
import itertools
from scipy.spatial import distance
warnings.simplefilter(action='ignore', category=FutureWarning)

## insertion

In [2]:
multichainLoc = ''
chainName = 'test_chain'
datadir = '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain/multichain'
dataPath = '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain/code/16k_scalability/data/vcf/wgs'
variantfile = 'all'
metaFile = '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain/code/16k_scalability/data/mapping_vocab/metadata.csv'
numberPeople = 100
person = int(numberPeople)
cpu = 1
chromosomes = 4
positions = '3416438'
genotypes = '1|1'
metadata = 'FreeBayes'
annotation_path = '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain/code/16k_scalability/data/annotations'

In [3]:
geneFile, variantFile, chrom = '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain/code/16k_scalability/data/gtf/gtf_chr4.txt', '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain/code/16k_scalability/data/vcf/wgs/chr_4.vcf.gz', 4
#read in gtf file using data
df = read_gtf(geneFile, 
                usecols=['seqname','gene_id','feature','start','end', 'gene_type', 'gene_name','strand'])

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name']


## PCA

### Insert

In [4]:
pcaFile = '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain/data/preprocessing/'
sampleIDFile = '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain/code/16k_scalability/data/vcf/wgs'

In [28]:
def loadData(pcaFile, sampleIDFile):
    '''
    Load sample principal components, reference loadings and sample ids
    Input:
        file paths for pca data and sample ids
    '''
    #Sample PCs
    sample_pcs = pd.read_csv(f'{pcaFile}pca/samples_pcs.csv')
    #Reference loadings
    ref_v = pd.read_csv(f'{pcaFile}pca_ref/v_loadings.csv')
    ref_center = pd.read_csv(f'{pcaFile}pca_ref/center_values.csv')
    ref_scale = pd.read_csv(f'{pcaFile}pca_ref/scale_values.csv')
    #Set sample IDs
    command = f'bcftools query -l {sampleIDFile}/chr_1.vcf.gz'
    items = subprocess.check_output(command.split())
    samples = items.decode().split("\n")[:-1]
    sample_pcs.index= samples
    return sample_pcs, (ref_v, ref_center, ref_scale)

In [191]:
def publishSamplePC(chainName, multichainLoc, datadir, pcaFile, sampleIDFile):
    '''
    Insert samples principal components
    Input:
        file paths for pca data and sample ids
    '''
    sample_pcs, ref_data = loadData(pcaFile, sampleIDFile)
    for i, row in sample_pcs.iterrows():
        streamName = 'GWAS'
        streamKeys = ['PCA', i]
        streamValues ='{"json":'+row.to_json()+'}' #create JSON and remove brackets

        publishCommand = [multichainLoc+'multichain-cli', 
            str('{}'.format(chainName)), 
            str('-datadir={}'.format(datadir)),
            'publish',
            str('{}'.format(streamName)), 
            str('["{}", "{}"]'.format(streamKeys[0], streamKeys[1])),
            str('{}'.format(streamValues))]
        procPublish = subprocess.Popen(publishCommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        procPublish.wait()

In [ ]:
publishSamplePC(chainName, multichainLoc, datadir, pcaFile, sampleIDFile)

### Query

In [201]:
sampleSearch = '91646,82285,37633'
kSearch = int('8')

In [203]:
def querySamplePCA(chainName, datadir, sampleSearch, kSearch):
    '''
    Query PC's of samples on chain
    Input:
        sampleSearch - sampled ids to extract
        kSearch - number of PCs to get
    '''
    #Extract data
    queryCommand = 'multichain-cli {} -datadir={} liststreamkeyitems GWAS {}'.format(chainName, datadir, "PCA")
    items = subprocess.check_output(queryCommand.split())
    matches = json.loads(items, parse_int= int)
    #Create DF
    sample_pcs = [(match_['keys'][1], match_['data']['json']) for match_ in matches]
    pc_df = pd.DataFrame.from_dict(dict(sample_pcs)).T
    #Filter based on search
    sampleSearch = sampleSearch.split(',')
    kSearch = int(kSearch)
    if sampleSearch != ['none']:
        pc_df = pc_df.loc[sampleSearch]
    if kSearch != ['all']:
        pc_df = pc_df.iloc[:,:kSearch]
    print(pc_df)
    return pc_df

In [204]:
querySamplePCA(chainName, datadir, sampleSearch, kSearch)

               V1          V2         V3         V4        V5        V6  \
91646 -111.040673 -182.611776 -14.214253 -46.936272 -6.919233  1.603086   
82285 -110.464685 -185.844757 -17.063498 -46.617285 -8.039273  0.601743   
37633 -109.987731 -185.830888 -13.367774 -47.066450 -7.891876  0.291234   

              V7        V8  
91646 -66.650671  5.665950  
82285 -68.136585  7.958371  
37633 -70.303100  7.115474  


,V1,V2,V3,V4,V5,V6,V7,V8
91646,-111.040673,-182.611776,-14.214253,-46.936272,-6.919233,1.603086,-66.650671,5.665950
82285,-110.464685,-185.844757,-17.063498,-46.617285,-8.039273,0.601743,-68.136585,7.958371
37633,-109.987731,-185.830888,-13.367774,-47.066450,-7.891876,0.291234,-70.303100,7.115474


# Relatedness

### Insert

In [89]:
relatedFile = '/gpfs/commons/groups/gursoy_lab/aelhussein/blockchain/data/preprocessing/relatedness'

In [488]:
queryCommand = f"bcftools query -l {relatedFile}/merged.vcf.gz"
#Load genotypes
items = subprocess.check_output(queryCommand, shell=True)
sampleids = items.decode().split('\n')
sampleids = sampleids[0:30]

In [485]:
def getRelatednessSNP(sampleids, relatedFile):
    '''
    Extract the related snps used by GRAF
    Input:
        sampleids - list of sampleids to insert
        relatedFile - location of the vcf file with snps
    '''
    #SNPS
    gt_final = {}
    #sampleids = sampleids.split(',')
    for sampleid in sampleids:
        queryCommand = f"bcftools query -f '[%GT]\\n' -s {sampleid} {relatedFile}/merged.vcf.gz"
        #Load genotypes
        process = subprocess.Popen(queryCommand, shell=True, stdout=subprocess.PIPE)
        gt = []
        while True:
            line = process.stdout.readline().decode().strip()
            if not line:
                break
            gt.append(line)

        process.wait()

        gt_mapper = {'0|0':0, '0|1':1, '1|0':1, '1|1':2}
        #Update gt
        modified_gt = []
        for item in gt:
            x, y = item.split('|')
            x = min(int(x), 1)
            y = min(int(y), 1)
            modified_gt.append(f'{x}|{y}')
        #Create final list per sample
        gt_final[sampleid] = [gt_mapper[c] for c in modified_gt]

    #AF
    queryCommand = f"bcftools query -f '%AF\n'  {relatedFile}/merged.vcf.gz"
    #Load AF's
    process = subprocess.Popen(queryCommand, shell=True, stdout=subprocess.PIPE)
    af = []
    while True:
        line = process.stdout.readline().decode().strip()
        if not line:
            break
        af.append(line)
    process.wait()
    af = [a[:4] for a in af]
    return gt_final, af


In [489]:
gt_final, af = getRelatednessSNP(sampleids, relatedFile)

In [319]:
def publishRelatednessSNP(chainName, datadir, multichainLoc, sampleids, relatedFile):
    '''
    Publish the related snp gentoypes per sample onto the chain
    Input:
        sampleids - list of sampleids to insert
        relatedFile - location of the vcf file with snps
    '''
    gt_final, af = getRelatednessSNP(sampleids, relatedFile)
    #SNPs
    for sample in gt_final:
        streamName = 'GWAS'
        streamKeys = ['Relatedness', sample]
        streamValues ='{"json":'+json.dumps(gt_final[sample])+'}' #create JSON and remove brackets
        publishCommand = [multichainLoc+'multichain-cli', 
            str('{}'.format(chainName)), 
            str('-datadir={}'.format(datadir)),
            'publish',
            str('{}'.format(streamName)), 
            str('["{}","{}"]'.format(streamKeys[0], streamKeys[1])),
            str('{}'.format(streamValues))]
        procPublish = subprocess.Popen(publishCommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        procPublish.wait()

    #AF
    streamName = 'GWAS'
    streamKeys = ['Relatedness', 'AF']
    streamValues ='{"json":'+json.dumps(af)+'}' #create JSON and remove brackets
    publishCommand = [multichainLoc+'multichain-cli', 
        str('{}'.format(chainName)), 
        str('-datadir={}'.format(datadir)),
        'publish',
        str('{}'.format(streamName)), 
        str('["{}","{}"]'.format(streamKeys[0], streamKeys[1])),
        str('{}'.format(streamValues))]
    procPublish = subprocess.Popen(publishCommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


In [320]:
publishRelatednessSNP(chainName, datadir, multichainLoc, sampleids, relatedFile)

### Query

In [341]:
def querySampleRelatedness(chainName, datadir, sampleSearch):
    '''
    Query relatedness snps from chain
    Input:
        sampleSearch - sampled ids to extract
    '''
    #Extract data
    queryCommand = 'multichain-cli {} -datadir={} liststreamkeyitems GWAS {}'.format(chainName, datadir, 'Relatedness')
    items = subprocess.check_output(queryCommand.split())
    matches = json.loads(items, parse_int= int)
    #Create DF
    sample_snps = [(match_['keys'][1], match_['data']['json']) for match_ in matches if match_['keys'][1].upper() != 'AF' ]
    rl_df = pd.DataFrame.from_dict(dict(sample_snps)).T
    #Filter based on search
    sampleSearch = sampleSearch.split(',')
    if sampleSearch != ['none']:
        rl_df = rl_df.loc[sampleSearch]

    #AF
    queryCommand = 'multichain-cli {} -datadir={} liststreamkeyitems GWAS {}'.format(chainName, datadir, 'AF')
    items = subprocess.check_output(queryCommand.split())
    matches = json.loads(items, parse_int= int)
    txid = matches[0]['data']['txid']
    queryCommand = 'multichain-cli {} -datadir={} gettxoutdata {} 0'.format(chainName, datadir, txid)
    items = subprocess.check_output(queryCommand.split())
    matches = json.loads(items, parse_int= int)['json']
    af = [float(match_) for match_ in matches]
    return rl_df, af

In [445]:
rl_df, af = querySampleRelatedness(chainName, datadir, sampleSearch)

In [562]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

def calculate_mismatch(df):
    n = df.shape[0] 
    agmr_matrix = np.zeros((n, n)) 
    hgmr_matrix = np.zeros((n, n)) 

    for i in range(n):
        for j in range(i+1, n): 
            x = df.iloc[i]  
            y = df.iloc[j]  
            
            # AGMR: percentage of SNPs on which the two genotypes are not identical
            agmr = np.mean(x != y)
            agmr_matrix[i, j] = agmr
            agmr_matrix[j, i] = agmr  
            
            # HGMR: genotype mismatch rate when only the SNPs with homozygous calls for both samples are considered
            homozygous_indices = (x % 2 == 0) & (y % 2 == 0)  
            hgmr = np.mean(x[homozygous_indices] != y[homozygous_indices]) if np.sum(homozygous_indices) > 0 else 0
            hgmr_matrix[i, j] = hgmr
            hgmr_matrix[j, i] = hgmr 

    
    agmr_df = pd.DataFrame(agmr_matrix, index=df.index, columns=df.index)
    hgmr_df = pd.DataFrame(hgmr_matrix, index=df.index, columns=df.index)

    return agmr_df, hgmr_df

def determine_relationships(agmr_df, hgmr_df, relationship_parameters):
    relationships = pd.DataFrame(index=agmr_df.index, columns=agmr_df.columns)
    
    for i in agmr_df.index:
        for j in agmr_df.columns:
            agmr = agmr_df.loc[i, j]
            hgmr = hgmr_df.loc[i, j]

            max_likelihood = -np.inf
            max_rel = None
            for rel, params in relationship_parameters.items():
                likelihood = multivariate_normal(mean=params['mean'], cov=params['cov']).pdf([agmr, hgmr])
                if likelihood > max_likelihood:
                    max_likelihood = likelihood
                    max_rel = rel
            relationships.loc[i, j] = max_rel
    return relationships

def assess_kinship(rl_df):
    relationship_parameters = {
        'ID': {'mean': [0.0, 0.0], 'cov': [[0.01**2, 0.01*0.38*0.317], [0.01*0.38*0.317, 0.38**2]]},
        'PO': {'mean': [0.04, 0.02], 'cov': [[0.07**2, 0.07*1.47*0.104], [ 0.07*1.47*0.104, 1.47**2]]},
        'FS': {'mean': [0.36, 0.1], 'cov': [[1.02**2,  1.02*2.32*0.784], [1.02*2.32*0.784, 2.32**2]]},
        'D2': {'mean': [0.40, 0.15], 'cov': [[1.35**2, 1.35*2.32*0.784], [0, 1.10**2]]},
        'D3': {'mean': [0.44, 0.19], 'cov': [[1.35**2, 1.35*0.96*0.830], [1.35*0.96*0.830, 0.96**2]]},
        'UR': {'mean': [0.54, 0.22], 'cov': [[1.35**2, 1.35*0.96*0.830], [1.35*0.96*0.830, 0.96**2]]}
    }
    agmr_df, hgmr_df  = calculate_mismatch(rl_df)
    relationships = determine_relationships(agmr_df, hgmr_df, relationship_parameters)
    return relationships

def queryKinship(chainName, datadir, sampleSearch):
    rl_df, af = querySampleRelatedness(chainName, datadir, sampleSearch)
    relationships = assess_kinship(rl_df)
    print(relationships)
    return relationships

In [563]:
queryKinship(chainName, datadir, sampleSearch)

        HG00108 HG00109 HG00110
HG00108      ID      UR      UR
HG00109      UR      ID      UR
HG00110      UR      UR      ID


,HG00108,HG00109,HG00110
HG00108,ID,UR,UR
HG00109,UR,ID,UR
HG00110,UR,UR,ID


In [520]:
def calculate_kinship(df, allele_frequencies):
    allele_frequencies = np.array(allele_frequencies)
    genotypes = df.values / 2
    p = allele_frequencies + 1e-2
    q = 1 - allele_frequencies
    def kinship(x, y):
        kinship_coefficient = np.mean((x - 2*p)*(y-2*p) / (2*p*q))
        return kinship_coefficient
    pairwise_kinship = distance.pdist(genotypes, kinship)
    square_kinship = distance.squareform(pairwise_kinship)
    kinship_df = pd.DataFrame(square_kinship, index=df.index, columns=df.index)
    #as squareform assumes 0 and not 1 for diagonal distance
    np.fill_diagonal(kinship_df.values, 1)
    return kinship_df.applymap(lambda x: round(x, 3))

In [524]:
def extractRelatedSampled(chainName, datadir, sampleSearch):
    #Calculate relatedness
    rl_df, af = querySampleRelatedness(chainName, datadir, sampleSearch)
    kinship = calculate_kinship(rl_df, af)
    #Identify related samples
    upper_triangular = np.triu(kinship.values, k=1)
    row_indices, col_indices = np.where(upper_triangular > 0.65)
    row_names = kinship.index[row_indices]
    col_names = kinship.columns[col_indices]
    related_samples = pd.DataFrame({'Sample_1': row_names, 'Sample_2': col_names})
    print(rl_df, kinship, related_samples)
    return rl_df, kinship, related_samples


In [525]:
rl_df, kinship, related_samples = extractRelatedSampled(chainName, datadir, sampleSearch)

         0     1     2     3     4     5     6     7     8     9     ...  \
HG00108     0     0     1     0     2     0     1     1     2     0  ...   
HG00109     0     0     0     1     0     0     1     2     1     0  ...   
HG00110     1     0     1     1     2     0     0     2     0     1  ...   

         9940  9941  9942  9943  9944  9945  9946  9947  9948  9949  
HG00108     1     1     0     2     2     0     1     0     1     0  
HG00109     1     0     0     2     2     0     0     2     1     0  
HG00110     2     2     0     1     2     0     0     0     1     0  

[3 rows x 9950 columns]          HG00108  HG00109  HG00110
HG00108    1.000    0.628    0.636
HG00109    0.628    1.000    0.631
HG00110    0.636    0.631    1.000 Empty DataFrame
Columns: [Sample_1, Sample_2]
Index: []


In [526]:
kinship

,HG00108,HG00109,HG00110
HG00108,1.000,0.628,0.636
HG00109,0.628,1.000,0.631
HG00110,0.636,0.631,1.000
